# Updated Tax-Calculator runs for FGA
This analysis includes:
- Static cost estimates (no behavioral response)
- Dynamic cost estimates (includes behavioral response)
- Distributional analyses

TODO:
- Add stacked analysis
- Fix behavioral response results in Section 3. The baseline of tcja permanence needs to be the behavioral response revenue relative to the current law baseline.

## 0. Setup

### 0.1. Update Tax-Calculator repository on your machine
First, you need to update the [`Tax-Calculator`](https://github.com/PSLmodels/Tax-Calculator) repository on your local machine using Git. You'll want to make sure to have an up-to-date `master` branch as well as a copy of Rick's `ingram_flat` (for flat tax) branch. Do the following steps. If you already have the Tax-Calculator repository forked and cloned, skip to step 4.

1. Make your own GitHub account on [GitHub.com](https://github.com/).
2. Fork the Tax-Calculator GitHub repository on your GitHub account. Go to the Tax-Calculator GitHub repository (https://github.com/PSLmodels/Tax-Calculator) in your browser and click the "Fork" button in the upper-right area of the screen. This will open a screen that confirms that you want to make a fork (an exact copy) of this repository on your GitHub account in the cloud.
3. Clone the repository from your GitHub account in the cloud (e.g., "`https://github.com/[YourGitHubHandle]/Tax-Calculator`" and NOT the "`PSLmodels`" URL) to your local computer. To clone this repository to your local machine:
    - In your terminal window (Windows Command Prompt or Mac Terminal), navigate to the directory where you want the repository to reside.
    - Type the following command: `git clone https://github.com/[YourGitHubHandle]/Tax-Calculator.git`, where the `[YourGitHubHandle]` is your GitHub handle
    - Change directory to inside the Tax-Calculator directory: `cd Tax-Calculator`
    - Make sure you have added an `upstream` remote to your repository. Type `git remote -v` to print a verbose list of remotes associated with your repository. It will always automatically have `origin` as a remote, which will be your fork `https://github.com/[YourGitHubHandle]/Tax-Calculator.git`. But we need to make sure there is also a remote named `upstream` which is the main Tax-Calculator repository `https://github.com/PSLmodels/Tax-Calculator.git`. If the `upstream` repository is not there, type: `git remote add upstream https://github.com/PSLmodels/Tax-Calculator.git`.
4. Update your Tax-Calculator `master` branch. Navigate to your Tax-Calculator repository in your terminal window and type the following three Git commands:
    - `git fetch upstream`
    - `git merge upstream/master`
    - `git push origin master`

### 0.2. Create the fga-taxcalc-dev conda environment
Do the following steps in order to ensure you have the most up-to-date `fga-taxcalc-dev` conda environment for running these analyses.

1. Delete your previous version of the `fga-taxcalc-dev` conda environment. Open your terminal window (Windows Command Prompt or Mac Terminal) and type: `conda remove --name fga-taxcalc-dev`.
2. Navigate to your `Tax-Calculator` repository directory, and switch to the `rickecon-flat` branch you pulled from Rick's branch: `git checkout rickecon-flat`
3. Create the `fga-taxcalc-dev` conda environment
    - Type: `conda env create -f environment-fga.yml`. This is a file that I saved in this branch, not for Tax-Calculator in general, but for running the analyses in this notebook.
    - Activate this environment: `conda activate fga-taxcalc-dev`
    - This environment has the taxcalc package installed from Rick's `ingram_flat` branch of his repository. So you should have all the changes you need to run the reforms.

### 0.3. Get Rick's updated versions of the TMD (tax-microdata-benchmarking) files
I don't know why the versions of the three TMD files that we created from the FGA version of the 2015 PUF are different than other files (in this example from Jason DeBacker). But the DeBacker files give the correct baseline revenues (`tmd_growfactors_jason.csv`, `tmd_jason.csv.gz`, and `tmd_weights_jason.csv.gz`). So I want to use those and figure out later why ours created a different set of files. I recommend storing these three files in the same `tax-microdata-benchmarking/tmd/storage/output` folder as we were using for the original files. Just leave the `_jason` suffixes on the files to designate than the ones that we created using the repository.

In [1]:
# Import libraries
import os
import numpy as np
import pandas as pd
from taxcalc import *
from taxcalc.growfactors import GrowFactors
from taxcalc.records import Records
from taxcalc.policy import Policy
from taxcalc.calculator import Calculator
import behresp
from taxbrain import (
    TaxBrain, differences_plot, distribution_plot, volcano_plot
)

import matplotlib.pyplot as plt
from bokeh.io import output_notebook, show

In [2]:
output_notebook()
pd.options.display.float_format = '{:,.3f}'.format

Loading BokehJS ...

In [4]:
# Set directory and file paths
MainFileDir = "/Users/richardevans/Docs/Economics/OSE/Federal/IngramFlatTax"
os.chdir(MainFileDir)  # Set path
print(os.getcwd())  # Print path to confirm
# Input the path to your TMD files on your local machine
tmd_dir = (
    "/Users/richardevans/Docs/Economics/OSE/microsim/" +
    "tax-microdata-benchmarking/tmd/storage/output"
)

/Users/richardevans/Docs/Economics/OSE/Federal/IngramFlatTax


In [5]:
def full_analysis(
    baseline_json, reform_json, tmd_data_dir, response_elasticities=None,
    bw_start_year=2026, bw_end_year=2035, baseline_name="", reform_name="",
    save_file_name_component="", years_dist_list=[2026, 2035]
):
    """
    Run a full analysis of baseline and reform tax policies using Tax-
    Caclulator

    Args:
        baseline_json (str): path to the baseline policy reform JSON file
        reform_json (str): path to the reform policy reform JSON file
        tmd_data_dir (str): path to the directory containing TMD data files
        response_elasticities (dict): dictionary of behavioral response
            elasticities for the tax policy reform
        bw_start_year (int): start year of the budget window
        bw_end_year (int): end year of the budget window
        baseline_name (str): name of the baseline policy
        reform_name (str): name of the reform policy
        save_file_name_component (str): component of the file name to use when
            saving the filer individual data
        years_dist_list (list): list of years for which to print distributional
            tables

    Returns:
        diag_tab1_list (list): list of diagnostic tables by each year in the
            budget window for the baseline policy
        diag_tab2_list (list): list of diagnostic tables for each year in the
            budget window for the reform policy
        df_dist_list (list): list of distributional tables for each year in the
            budget window for the baseline and reform policies
        df_diff_list (list): list of distributional difference tables for each
            year in the budget window for the baseline and reform policies
        df_filer_stat_list (list): list of filer individual dataframes for each
            year of the budget window for the baseline and reform policies with
            no behavioral response (static)
        df_filer_dyn_list (list): list of filer individual dataframes for each
            year of the budget window for the baseline and reform policies with
            behavioral response (dynamic)
    """
    # Create GrowFactors object that uses tmd_growfactors.csv
    gf_tmd_path = os.path.join(tmd_data_dir, "tmd_growfactors_jason.csv")
    gf_tmd = GrowFactors(gf_tmd_path)
    # Create Records object that uses tmd data, grow factors and weights
    recs_tmd = Records(
        data=os.path.join(tmd_data_dir, "tmd_jason.csv.gz"),
        start_year=Records.TMDCSV_YEAR,
        gfactors=gf_tmd,
        weights=os.path.join(tmd_data_dir, "tmd_weights_jason.csv.gz"),
        adjust_ratios=None
    )
    # Create baseline policy calculator object
    pol1 = Policy()
    if baseline_json is not None:
        pol1_dict = pol1.read_json_reform(baseline_json)
        pol1.implement_reform(pol1_dict)
    calc1 = Calculator(policy=pol1, records=recs_tmd)

    # Create reform policy calculator object
    if baseline_json is None:
        pol2 = Policy()
        pol2_dict = pol2.read_json_reform(reform_json)
        pol2.implement_reform(pol2_dict)
    else:
        pol2 = Policy()
        pol2_dict1 = pol2.read_json_reform(baseline_json)
        pol2.implement_reform(pol2_dict1)
        pol2_dict2 = pol2.read_json_reform(reform_json)
        pol2.implement_reform(pol2_dict2)
    calc2 = Calculator(policy=pol2, records=recs_tmd)

    # Create cost estimate and distributional objects for each year of the
    # budget window
    rev1 = {'IIT': {}, 'Payroll': {}}
    rev2_stat = {'IIT': {}, 'Payroll': {}}
    diag_tab1_list = []
    diag_tab2_list = []
    df_dist_list = []
    df_diff_list = []
    df_filer_stat_list = []
    if response_elasticities is not None:
        rev2_dyn = {'IIT': {}, 'Payroll': {}}
        df_filer_dyn_list = []

    for t in range(bw_start_year, bw_end_year + 1):
        # Set the year for each calculator
        calc1.advance_to_year(t)
        calc2.advance_to_year(t)

        # Calculate the baseline and reform taxes
        calc1.calc_all()
        calc2.calc_all()

        # Get the IIT and payroll tax total revenue data for the current year
        rev1['IIT'][t] = calc1.weighted_total('iitax')
        rev1['Payroll'][t] = calc1.weighted_total('payrolltax')
        rev2_stat['IIT'][t] = calc2.weighted_total('iitax')
        rev2_stat['Payroll'][t] = calc2.weighted_total('payrolltax')

        if response_elasticities is not None:
            _, df2br = behresp.response(calc1, calc2, response_elasticities)
            df_filer_dyn_list.append(df2br)
            rev2_dyn['IIT'][t] = (df2br['iitax'] * df2br['s006']).sum()
            rev2_dyn['Payroll'][t] = (
                df2br['payrolltax'] * df2br['s006']
            ).sum()

        # Create two diagnostic tables
        diag_tab1_list.append(calc1.diagnostic_table(1))
        diag_tab2_list.append(calc2.diagnostic_table(1))

        # Create distribution level and distribution difference tables
        dist_tab1, dist_tab2 = calc1.distribution_tables(
            calc2, 'weighted_deciles'
        )
        assert isinstance(dist_tab1, pd.DataFrame)
        assert isinstance(dist_tab2, pd.DataFrame)
        df_dist = pd.DataFrame()
        df_dist['funits(#m)'] = dist_tab1['count']
        df_dist['itax1($b)'] = dist_tab1['iitax']
        df_dist['itax2($b)'] = dist_tab2['iitax']
        df_dist['aftertax_inc1($b)'] = dist_tab1['aftertax_income']
        df_dist['aftertax_inc2($b)'] = dist_tab2['aftertax_income']
        df_dist_list.append(df_dist)

        diff_table = calc1.difference_table(
            calc2, 'weighted_deciles', 'iitax'
        )
        assert isinstance(diff_table, pd.DataFrame)
        df_diff = pd.DataFrame()
        dif_colnames = ['count', 'tot_change', 'mean', 'pc_aftertaxinc']
        ext_colnames = [
            'funits(#m)', 'agg_diff($b)', 'mean_diff($)', 'aftertaxinc_diff(%)'
        ]
        for dname, ename in zip(dif_colnames, ext_colnames):
            df_diff[ename] = diff_table[dname]
        df_diff_list.append(df_diff)

        # Create a DataFram with all the filer observations iitax for baseline
        # reform, and the difference in dollars and percent
        df_filer_stat = pd.DataFrame()
        df_filer_stat['baseline expanded income($)'] = calc1.array(
            'expanded_income'
        )
        df_filer_stat['iitax_1($)'] = calc1.array('iitax')
        df_filer_stat['iitax_2($)'] = calc2.array('iitax')
        df_filer_stat['iitax_diff($)'] = (
            df_filer_stat['iitax_2($)'] -
            df_filer_stat['iitax_1($)']
        )
        df_filer_stat['iitax_diff(%)'] = (
            df_filer_stat['iitax_diff($)'] /
            df_filer_stat['iitax_1($)']
        )
        df_filer_stat['pop weight'] = calc1.array('s006')
        df_filer_stat_list.append(df_filer_stat)
        if response_elasticities is not None:
            df_filer_dyn_list.append(df2br)

    # make table of revenue estimates
    rev1_df = pd.DataFrame.from_dict(rev1).T
    rev2_stat_df = pd.DataFrame.from_dict(rev2_stat).T
    diff_stat_df = (rev2_stat_df - rev1_df) * 1e-9
    diff_stat_df[
        str(bw_start_year) + '-' + str(bw_end_year)
    ] = diff_stat_df.sum(axis=1)
    if response_elasticities is not None:
        rev2_dyn_df = pd.DataFrame.from_dict(rev2_dyn).T
        diff_dyn_df = (rev2_dyn_df - rev1_df) * 1e-9
        diff_dyn_df[
            str(bw_start_year) + '-' + str(bw_end_year)
        ] = diff_dyn_df.sum(axis=1)

    # Print static and dynamic budget window revenue estimate tables
    pd.options.display.float_format = '${:.3f}'.format
    print(
        reform_name + ' vs. ' + baseline_name + ' revenue estimates: ' +
        'static, ' + str(int(bw_start_year)) + '-' + str(int(bw_end_year))
    )
    print(diff_stat_df.to_markdown())
    if response_elasticities is not None:
        print("")
        print(
            reform_name + ' vs. ' + baseline_name + ' revenue estimates: ' +
            'dynamic (behavioral response), ' + str(int(bw_start_year)) + '-' +
            str(int(bw_end_year))
        )
        print(diff_dyn_df.to_markdown())

    if len(years_dist_list) > 0:
        for year in years_dist_list:
            # Print diagnostic table
            print("")
            print(
                baseline_name + " static diagnostic table for year " +
                str(year)
            )
            print(diag_tab1_list[year - bw_start_year])
            print("")
            print(
                reform_name + " static diagnostic table for year " +
                str(year)
            )
            print(diag_tab2_list[year - bw_start_year])
            # Print distributional table in levels
            print("")
            print("Distributional table in levels for year " + str(year))
            print(df_dist_list[year - bw_start_year])
            # Print distributional table in differences
            print("")
            print("Distributional table in differences for year " + str(year))
            print(df_diff_list[year - bw_start_year])
            # Save filer individual data
            df_filer_stat_list[year - bw_start_year].to_csv(
                os.path.join(
                    "./data",
                    'df_filer_stat_' + save_file_name_component + str(year) +
                    '.csv'
                ),
                index=False
            )
            print("")
            print("Saved filer individual static data for year " + str(year))
            df_filer_stat_list[year - bw_start_year]
            if response_elasticities is not None:
                # Save filer individual data
                df_filer_dyn_list[year - bw_start_year].to_csv(
                    os.path.join(
                        "./data",
                        'df_filer_dyn_' + save_file_name_component +
                        str(year) +'.csv'
                    ),
                    index=False
                )

    return (
        diag_tab1_list, diag_tab2_list, df_dist_list, df_diff_list,
        df_filer_stat_list, df_filer_dyn_list
    )

In [ ]:
# def stacked_analysis(
#     reforms_list, reforms_str_list, tmd_data_dir, start_with_currentlaw = True,
#     response_elasticities=None, bw_start_year=2026, bw_end_year=2035
# ):
#     """
#     Run a stacked static or behavioral response analysis of changes in total
#     individual income tax liability for a consecutive series of reforms.

#     Args:
#         reforms_list (list): list of reform policy dictionary files
#         reforms_str_list (list): list of string descriptors of reform policies
#         tmd_data_dir (str): path to the directory containing TMD data files
#         start_with_currentlaw (bool): if True, the first reform in the list
#             is the current law policy; if False, the first reform in the list
#             is the first reform policy
#         response_elasticities (None or dict): dictionary of behavioral response
#             elasticities for the tax policy reform
#         bw_start_year (int): start year of the budget window
#         bw_end_year (int): end year of the budget window

#     Returns:
#         diag_tab1_list (list): list of diagnostic tables by each year in the
#             budget window for the baseline policy
#         diag_tab2_list (list): list of diagnostic tables for each year in the
#             budget window for the reform policy
#         df_dist_list (list): list of distributional tables for each year in the
#             budget window for the baseline and reform policies
#         df_diff_list (list): list of distributional difference tables for each
#             year in the budget window for the baseline and reform policies
#         df_filer_stat_list (list): list of filer individual dataframes for each
#             year of the budget window for the baseline and reform policies with
#             no behavioral response (static)
#         df_filer_dyn_list (list): list of filer individual dataframes for each
#             year of the budget window for the baseline and reform policies with
#             behavioral response (dynamic)
#     """
#     if start_with_currentlaw:
#         bases_list = [{}] + reforms_list[:-1]
#     else:
#         bases_list = reforms_list[:-1]
#         reforms_list = reforms_list[1:]
#         reforms_str_list = reforms_str_list[1:]
#     # Create GrowFactors object that uses tmd_growfactors.csv
#     gf_tmd_path = os.path.join(tmd_data_dir, "tmd_growfactors_jason.csv")
#     gf_tmd = GrowFactors(gf_tmd_path)
#     # Create Records object that uses tmd data, grow factors and weights
#     recs_tmd = Records(
#         data=os.path.join(tmd_data_dir, "tmd_jason.csv.gz"),
#         start_year=Records.TMDCSV_YEAR,
#         gfactors=gf_tmd,
#         weights=os.path.join(tmd_data_dir, "tmd_weights_jason.csv.gz"),
#         adjust_ratios=None
#     )
#     # Create baseline policy calculator object
#     pol1 = Policy()
#     if baseline_json is not None:
#         pol1_dict = pol1.read_json_reform(baseline_json)
#         pol1.implement_reform(pol1_dict)
#     calc1 = Calculator(policy=pol1, records=recs_tmd)

#     # Create reform policy calculator object
#     if baseline_json is None:
#         pol2 = Policy()
#         pol2_dict = pol2.read_json_reform(reform_json)
#         pol2.implement_reform(pol2_dict)
#     else:
#         pol2 = Policy()
#         pol2_dict1 = pol2.read_json_reform(baseline_json)
#         pol2.implement_reform(pol2_dict1)
#         pol2_dict2 = pol2.read_json_reform(reform_json)
#         pol2.implement_reform(pol2_dict2)
#     calc2 = Calculator(policy=pol2, records=recs_tmd)

#     # Create cost estimate and distributional objects for each year of the
#     # budget window
#     rev1 = {'IIT': {}, 'Payroll': {}}
#     rev2_stat = {'IIT': {}, 'Payroll': {}}
#     diag_tab1_list = []
#     diag_tab2_list = []
#     df_dist_list = []
#     df_diff_list = []
#     df_filer_stat_list = []
#     if response_elasticities is not None:
#         rev2_dyn = {'IIT': {}, 'Payroll': {}}
#         df_filer_dyn_list = []

#     for t in range(bw_start_year, bw_end_year + 1):
#         # Set the year for each calculator
#         calc1.advance_to_year(t)
#         calc2.advance_to_year(t)

#         # Calculate the baseline and reform taxes
#         calc1.calc_all()
#         calc2.calc_all()

#         # Get the IIT and payroll tax total revenue data for the current year
#         rev1['IIT'][t] = calc1.weighted_total('iitax')
#         rev1['Payroll'][t] = calc1.weighted_total('payrolltax')
#         rev2_stat['IIT'][t] = calc2.weighted_total('iitax')
#         rev2_stat['Payroll'][t] = calc2.weighted_total('payrolltax')

#         if response_elasticities is not None:
#             _, df2br = behresp.response(calc1, calc2, response_elasticities)
#             df_filer_dyn_list.append(df2br)
#             rev2_dyn['IIT'][t] = (df2br['iitax'] * df2br['s006']).sum()
#             rev2_dyn['Payroll'][t] = (
#                 df2br['payrolltax'] * df2br['s006']
#             ).sum()

#         # Create two diagnostic tables
#         diag_tab1_list.append(calc1.diagnostic_table(1))
#         diag_tab2_list.append(calc2.diagnostic_table(1))

#         # Create distribution level and distribution difference tables
#         dist_tab1, dist_tab2 = calc1.distribution_tables(
#             calc2, 'weighted_deciles'
#         )
#         assert isinstance(dist_tab1, pd.DataFrame)
#         assert isinstance(dist_tab2, pd.DataFrame)
#         df_dist = pd.DataFrame()
#         df_dist['funits(#m)'] = dist_tab1['count']
#         df_dist['itax1($b)'] = dist_tab1['iitax']
#         df_dist['itax2($b)'] = dist_tab2['iitax']
#         df_dist['aftertax_inc1($b)'] = dist_tab1['aftertax_income']
#         df_dist['aftertax_inc2($b)'] = dist_tab2['aftertax_income']
#         df_dist_list.append(df_dist)

#         diff_table = calc1.difference_table(
#             calc2, 'weighted_deciles', 'iitax'
#         )
#         assert isinstance(diff_table, pd.DataFrame)
#         df_diff = pd.DataFrame()
#         dif_colnames = ['count', 'tot_change', 'mean', 'pc_aftertaxinc']
#         ext_colnames = [
#             'funits(#m)', 'agg_diff($b)', 'mean_diff($)', 'aftertaxinc_diff(%)'
#         ]
#         for dname, ename in zip(dif_colnames, ext_colnames):
#             df_diff[ename] = diff_table[dname]
#         df_diff_list.append(df_diff)

#         # Create a DataFram with all the filer observations iitax for baseline
#         # reform, and the difference in dollars and percent
#         df_filer_stat = pd.DataFrame()
#         df_filer_stat['baseline expanded income($)'] = calc1.array(
#             'expanded_income'
#         )
#         df_filer_stat['iitax_1($)'] = calc1.array('iitax')
#         df_filer_stat['iitax_2($)'] = calc2.array('iitax')
#         df_filer_stat['iitax_diff($)'] = (
#             df_filer_stat['iitax_2($)'] -
#             df_filer_stat['iitax_1($)']
#         )
#         df_filer_stat['iitax_diff(%)'] = (
#             df_filer_stat['iitax_diff($)'] /
#             df_filer_stat['iitax_1($)']
#         )
#         df_filer_stat['pop weight'] = calc1.array('s006')
#         df_filer_stat_list.append(df_filer_stat)
#         if response_elasticities is not None:
#             df_filer_dyn_list.append(df2br)

#     # make table of revenue estimates
#     rev1_df = pd.DataFrame.from_dict(rev1).T
#     rev2_stat_df = pd.DataFrame.from_dict(rev2_stat).T
#     diff_stat_df = (rev2_stat_df - rev1_df) * 1e-9
#     diff_stat_df[
#         str(bw_start_year) + '-' + str(bw_end_year)
#     ] = diff_stat_df.sum(axis=1)
#     if response_elasticities is not None:
#         rev2_dyn_df = pd.DataFrame.from_dict(rev2_dyn).T
#         diff_dyn_df = (rev2_dyn_df - rev1_df) * 1e-9
#         diff_dyn_df[
#             str(bw_start_year) + '-' + str(bw_end_year)
#         ] = diff_dyn_df.sum(axis=1)

#     # Print static and dynamic budget window revenue estimate tables
#     pd.options.display.float_format = '${:.3f}'.format
#     print(
#         reform_name + ' vs. ' + baseline_name + ' revenue estimates: ' +
#         'static, ' + str(int(bw_start_year)) + '-' + str(int(bw_end_year))
#     )
#     print(diff_stat_df.to_markdown())
#     if response_elasticities is not None:
#         print("")
#         print(
#             reform_name + ' vs. ' + baseline_name + ' revenue estimates: ' +
#             'dynamic (behavioral response), ' + str(int(bw_start_year)) + '-' +
#             str(int(bw_end_year))
#         )
#         print(diff_dyn_df.to_markdown())

#     if len(years_dist_list) > 0:
#         for year in years_dist_list:
#             # Print diagnostic table
#             print("")
#             print(
#                 baseline_name + " static diagnostic table for year " +
#                 str(year)
#             )
#             print(diag_tab1_list[year - bw_start_year])
#             print("")
#             print(
#                 reform_name + " static diagnostic table for year " +
#                 str(year)
#             )
#             print(diag_tab2_list[year - bw_start_year])
#             # Print distributional table in levels
#             print("")
#             print("Distributional table in levels for year " + str(year))
#             print(df_dist_list[year - bw_start_year])
#             # Print distributional table in differences
#             print("")
#             print("Distributional table in differences for year " + str(year))
#             print(df_diff_list[year - bw_start_year])
#             # Save filer individual data
#             df_filer_stat_list[year - bw_start_year].to_csv(
#                 os.path.join(
#                     "./data",
#                     'df_filer_stat_' + save_file_name_component + str(year) +
#                     '.csv'
#                 ),
#                 index=False
#             )
#             print("")
#             print("Saved filer individual static data for year " + str(year))
#             df_filer_stat_list[year - bw_start_year]
#             if response_elasticities is not None:
#                 # Save filer individual data
#                 df_filer_dyn_list[year - bw_start_year].to_csv(
#                     os.path.join(
#                         "./data",
#                         'df_filer_dyn_' + save_file_name_component +
#                         str(year) +'.csv'
#                     ),
#                     index=False
#                 )

#     return (
#         diag_tab1_list, diag_tab2_list, df_dist_list, df_diff_list,
#         df_filer_stat_list, df_filer_dyn_list
#     )

## 1. Evaluate the cost of TCJA permanence relative to current law baseline
This takes about two-and-a-half minutes to run on Rick's computer. Select to see the output as a "Scrollable element".

In [6]:
baseline_json_path = None
reform_json_path = os.path.join(MainFileDir, "json", "TCJA_ext.json")
# Specify response elasticities for behavioral response estimates
elasticities = {'sub': 0.25, 'inc': -0.5, 'cg': -0.8}
(
    diag_tab_curlaw_list, diag_tab_tcjaperm_list, df_dist_tcjaperm_curlaw_list,
    df_diff_tjcaperm_curlaw_list, df_filer_stat_tcjaperm_curlaw_list,
    df_filer_dyn_tcjaperm_curlaw_list
) = full_analysis(
    baseline_json_path,
    reform_json_path,
    tmd_dir,
    response_elasticities=elasticities,
    bw_start_year=2026,
    bw_end_year=2035,
    baseline_name="Current Law",
    reform_name="TCJA Permanence",
    save_file_name_component="tcjaperm_curlaw",
    years_dist_list=[2026, 2035]
)

TCJA Permanence vs. Current Law revenue estimates: static, 2026-2035
|         |     2026 |    2027 |     2028 |     2029 |     2030 |     2031 |     2032 |     2033 |     2034 |     2035 |   2026-2035 |
|:--------|---------:|--------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|------------:|
| IIT     | -311.536 | -320.74 | -329.846 | -323.457 | -332.822 | -342.295 | -351.736 | -361.213 | -370.695 | -380.339 |    -3424.68 |
| Payroll |    0     |    0    |    0     |    0     |    0     |    0     |    0     |    0     |    0     |    0     |        0    |

TCJA Permanence vs. Current Law revenue estimates: dynamic (behavioral response), 2026-2035
|         |        2026 |        2027 |        2028 |       2029 |       2030 |       2031 |       2032 |       2033 |       2034 |       2035 |   2026-2035 |
|:--------|------------:|------------:|------------:|-----------:|-----------:|-----------:|-----------:|-----------:|-----------:|--------

## 2. Evaluate the cost of the Ingram flat tax relative to current law baseline
This policy does not change any of the above-the-line deductions in calculating US tax liability and would do the following:
- Set all marginal individual income tax rates to 20%
- Set all pass-through individual income tax rates to 20%
- Set all the capital gains tax rates to 20%
- Set the AMT individual income tax rates to 0% and the AMT capital gains tax rates to 0%
- Increase the Standard deduction to $50k for joint filers and $25K for all other filers. Set the aged standard deduction to $0 for all filers.
- Create a Per child exemption of $20k for all filers.
- Get rid of the EITC, CTC, and ACTC credits.

This takes about two-and-a-half minutes to run on Rick's computer. Select to see the output as a "Scrollable element".

In [7]:
baseline_json_path = None
reform_json_path = os.path.join(MainFileDir, "json", "Ingram_flat.json")
(
    diag_tab_curlaw_list, diag_tab_ingram_list, df_dist_ingram_curlaw_list,
    df_diff_ingram_curlaw_list, df_filer_stat_ingram_curlaw_list,
    df_filer_dyn_ingram_curlaw_list
) = full_analysis(
    baseline_json_path,
    reform_json_path,
    tmd_dir,
    response_elasticities=elasticities,
    bw_start_year=2026,
    bw_end_year=2035,
    baseline_name="Current Law",
    reform_name="Ingram Flat Tax",
    save_file_name_component="ingram_curlaw",
    years_dist_list=[2026, 2035]
)

Ingram Flat Tax vs. Current Law revenue estimates: static, 2026-2035
|         |     2026 |     2027 |     2028 |     2029 |     2030 |     2031 |     2032 |     2033 |     2034 |     2035 |   2026-2035 |
|:--------|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|------------:|
| IIT     | -286.747 | -301.954 | -318.167 | -332.503 | -350.582 | -369.263 | -388.843 | -408.402 | -428.476 | -449.294 |    -3634.23 |
| Payroll |    0     |    0     |    0     |    0     |    0     |    0     |    0     |    0     |    0     |    0     |        0    |

Ingram Flat Tax vs. Current Law revenue estimates: dynamic (behavioral response), 2026-2035
|         |      2026 |      2027 |      2028 |      2029 |      2030 |      2031 |      2032 |      2033 |      2034 |      2035 |   2026-2035 |
|:--------|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|----------:|------------:|
| 

## 3. Evaluate the cost of the Ingram flat tax relative to TCJA permanence baseline
This policy does not change any of the above-the-line deductions in calculating US tax liability and would do the following:
- Set all marginal individual income tax rates to 20%
- Set all pass-through individual income tax rates to 20%
- Set all the capital gains tax rates to 20%
- Set the AMT individual income tax rates to 0% and the AMT capital gains tax rates to 0%
- Increase the Standard deduction to $50k for joint filers and $25K for all other filers. Set the aged standard deduction to $0 for all filers.
- Create a Per child exemption of $20k for all filers.
- Get rid of the EITC, CTC, and ACTC credits.

This takes about two-and-a-half minutes to run on Rick's computer. Select to see the output as a "Scrollable element".

In [8]:
baseline_json_path = os.path.join(MainFileDir, "json", "TCJA_ext.json")
reform_json_path = os.path.join(MainFileDir, "json", "Ingram_flat.json")
(
    diag_tab_tcjaperm_list, diag_tab_ingram_list, df_dist_ingram_tcjaperm_list,
    df_diff_ingram_tcjaperm_list, df_filer_stat_ingram_tcjaperm_list,
    df_filer_dyn_ingram_tcjaperm_list
) = full_analysis(
    baseline_json_path,
    reform_json_path,
    tmd_dir,
    response_elasticities=elasticities,
    bw_start_year=2026,
    bw_end_year=2035,
    baseline_name="TCJA Permanence",
    reform_name="Ingram Flat Tax",
    save_file_name_component="ingram_tcjaperm",
    years_dist_list=[2026, 2035]
)

Ingram Flat Tax vs. TCJA Permanence revenue estimates: static, 2026-2035
|         |     2026 |     2027 |     2028 |     2029 |     2030 |     2031 |    2032 |     2033 |     2034 |     2035 |   2026-2035 |
|:--------|---------:|---------:|---------:|---------:|---------:|---------:|--------:|---------:|---------:|---------:|------------:|
| IIT     | -40.1402 | -48.6491 | -58.2641 | -69.0958 | -80.1154 | -91.6846 | -104.25 | -116.831 | -129.965 | -143.769 |    -882.763 |
| Payroll |   0      |   0      |   0      |   0      |   0      |   0      |    0    |    0     |    0     |    0     |       0     |

Ingram Flat Tax vs. TCJA Permanence revenue estimates: dynamic (behavioral response), 2026-2035
|         |    2026 |     2027 |    2028 |     2029 |     2030 |     2031 |     2032 |     2033 |     2034 |     2035 |   2026-2035 |
|:--------|--------:|---------:|--------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|------------:|
| IIT     | 11.3334 | 